# Прогнозирование и оптимизация складских остатков

Базовая кафедра: Кафедра логистических систем и технологий  
Обучающийся: Шмунь М.С., Б03-907а  
Научный руководитель: Старостин Е.А.

## Введение

Доля Интернет-торговли в розничной торговле ежегодно увеличиваются: по данным Росстата (табл. 1), в 2022 году 6% от общего объема приходилась на продажи через Интернет. При этом сохраняются все предпосылки для дальнейшего развития этого тренда: по мере распространения и улучшения качества сервиса растет как доверие населения к совершению покупок через Интернет, так и интерес предпринимателей к реализации своих товаров и услуг в сети. Для сравнения, в Китае, где проникновение цифровых сервисов в жизнь граждан и их взаимная интеграция находится на куда более развитом уровне, эта доля в 2020 году составляла 30%. (рис. 2). 
Увеличивающийся объем Интернет-торговли приводит к более высокому уровню конкуренции на разных уровнях экономики, как среди отдельных продавцов, так и среди площадок: покупателю практически мгновенно доступен широкий ассортимент товаров, а также открытая информация о ценах и отзывы об уровне сервиса и качестве товаров, что вынуждает поставщиков поддерживать конкурентные уровни цен и качества обслуживания. Кроме того, в условиях нарастающих объемов даже небольшие оптимизации в операционных процессах могут стать значимым конкурентным преимуществом и ключом к росту доли продавца или площадки на рынке.
В этой работе были изучены и реализованы подходы по оптимизации управления складскими остатками на основе прогнозирования, которые могут быть использованы как отдельными продавцами-участниками электронной коммерции, так и площадками (маркетплейсами) для постановки целей поставщикам. Основной моделью для построения прогноза является ARIMA (англ. Auto-Regressive Integrated Moving Average), а также ее модификации с сезонными и экзогенными факторами.

Модели были построены на основе данных о фактических продажах товаров в десяти различных товарных категориях в кластере распределительного центра в Свердловской области за 2021-2022 годы и 1 квартал 2023 года. 
Модели оценивались как по статистическим показателям (MAPE, RMSE) так и по бизнес-показателям, рассчитанным апостериорно по результатам моделирования: упущенная выручка, затраты на хранение излишков, уровень сервиса. Таким образом, оценивается не только синтетическая «точность» прогноза, но и влияние от внедрения модели на успешность предприятия, что может быть полезно для оценки подхода на соответствие целям и задачам бизнеса и принятия решения о внедрении той или иной модели. 

## Обзор данных

Основной набор данных представляет из себя таблицу с понедельным срезом продаж (в штуках и в рублях) со следующими столбцами: неделя (по понедельнику), название категории, продажи в рублях, продажи в штуках.
Кроме того, для моделей, учитывающих экзогенные факторы, предоставлены периоды распродаж и даты аномалий на макроуровне (начало СВО, мобилизация). Для каждой категории также указаны норма (в процентах от стоимости) прибыли площадки за реализацию и затрат на хранение одной товарной единицы.


## Обзор классических подходов

### Экспертные модели

TODO

### Модель экономичного размера заказа
Распространенной моделью управления товарными остатками является EOQ (англ. Economic Order Quantity – экономичный размер заказа). В этой модели заданы следующие условия: 
	Спрос на товары известен и постоянен
	Размер заказа не ограничен
	Доставка товара на склад производится мгновенно после заказа

В постановке задачи EOQ введены следующие обозначения: 
	d – фиксированный спрос за период
	с – цена продажи единицы товара;
	К – затраты на размещение заказа;
	h — издержки за хранение единицы товара на складе;
	Q – искомая величина заказа

Решение задачи известно и детерминировано:
$$ Q^*=\sqrt{(\frac{2Kd}{h})} $$

При этом заказ производится при обнулении остатка.

Такое сильное ограничение как постоянный известный спрос сильно ограничивает применимость модели EOQ в реальных задачах, поэтому сейчас такой подход считается устаревшим, и чаще применяются модели, где спрос представлен либо детерминированной функции от внешних переменных (время, цена и т.д.), либо случайный


### Модели с фиксированными параметрами

К таким моделям управления остатками можно отнести модели с фиксированными промежутками между заказами (где поставщик управляет размером заказа) и с фиксированным размером заказа (где поставщик управляет моментом оформления заказа). Такие модели применяются в случае, если ограничения параметров обусловлены внешними факторами, например условиями работы производителя, возможностями логистической сети и т.д.


## Модели на основе прогнозирования спроса

Для моделей на основе прогнозирования спрос в периоде полагается случайной величиной с некоторой (не обязательно известной) функцией распределения, а сам ряд измерений спроса представляется экземпляром случайным процессом. Для такой задачи также характерен равный промежуток между заказами, например, минимально возможный, а также мгновенная доставка. В таких условиях очевидна стратегия поведения при идеальном прогнозировании: размещать заказ в точности равный спросу в ближайшем периоде. Таким образом полностью покрывается потребительский спрос, что максимизирует полученную выручку, и минимизируются затраты на хранение.  

В реальности же прогнозирование спроса на товары является сложной задачей: временные ряды такой природы редко являются стационарными, а также подвержены влиянию сезонных и внешних факторов. Для повышения качества прогнозирования применяются самые различные подходы: как в подготовке данных (логарифмирование, взятие разностей, очистка от выбросов и пропусков данных), так и в архитектуре прогнозных моделей (учет сезонности и тренда, авторегрессия, использования рекуррентных и сверточных нейронных сетей).  

При этом ошибка прогнозирования влияет на прибыль ассиметрично: при избыточном запасе предприятие теряет только стоимость хранения излишка, а при недостаточном – всю выручку от реализации непроданных единиц и связанную с этим прибыль. В условиях высокой конкуренции продавцов покупатель, обнаружив дефицит товара у поставщика, скорее всего просто купит его у конкурента (а конкурент, обнаружив дефицит – повысит цены и получит дополнительную прибыль от реализации).  

Таким образом, в процессе управления товарными запасами возникает необходимость не только прогнозировать спрос (результатом прогноза обычно является среднее или наиболее вероятное значение случайной величины), но и интерпретировать полученный прогноз для минимизации суммарных потерь или достижения других целевых показателей: например, использовать верхние границы доверительного интервала для обеспечения заданной нормы уровня сервиса и минимизации упущенной прибыли.


Простейшей моделью для прогнозирования спроса можно считать скользящее среднее (SMA – simple moving average, простое скользящее среднее) где прогнозом на период считается среднее значение нескольких предыдущих периодов

$$ SMA_i= \frac{1}{n} \sum_{k=1}^{N} D_{(i-k)} $$

